In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

FOLDERNAME = 'ENPM703/project/Final'
assert FOLDERNAME is not None, "[!] Enter the foldername."

import sys
sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))

Mounted at /content/drive


In [ ]:
import os
import shutil
from sklearn.model_selection import train_test_split

base_dir = "/content/drive/MyDrive/ENPM703/Project/Final/Dataset3/Individual_Participant"
output_dir = "/content/drive/MyDrive/ENPM703/Project/Final/Dataset3/Dataset_Split3"
categories = ["TC", "TS"]

participants = []
labels = []

for category in categories:
    category_path = os.path.join(base_dir, category)
    if os.path.exists(category_path):
        for participant_id in os.listdir(category_path):
            participants.append((category, participant_id))
            labels.append(category)

train_ids, temp_ids, train_labels, temp_labels = train_test_split(
    participants, labels, test_size=0.3, stratify=labels, random_state=7
)
val_ids, test_ids, val_labels, test_labels = train_test_split(
    temp_ids, temp_labels, test_size=0.5, stratify=temp_labels, random_state=7
)

def copy_participants(participant_list, split_name):
    for category, participant_id in participant_list:
        src_dir = os.path.join(base_dir, category, participant_id)
        dest_dir = os.path.join(output_dir, split_name, category, participant_id)
        if not os.path.exists(dest_dir):
            os.makedirs(dest_dir)

        for file_name in os.listdir(src_dir):
            src_file = os.path.join(src_dir, file_name)
            dest_file = os.path.join(dest_dir, file_name)
            shutil.copy(src_file, dest_file)

for split_name, participant_list in zip(
    ["Train", "Validation", "Test"], [train_ids, val_ids, test_ids]
):
    copy_participants(participant_list, split_name)

print("Dataset split into train, validation, and test folders with required structure!")


Dataset split into train, validation, and test folders with required structure!


In [2]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img

train_dir = "/content/drive/MyDrive/ENPM703/Project/Final/Dataset3/Dataset_Split4/Train"
augmented_dir = "/content/drive/MyDrive/ENPM703/Project/Final/Dataset3/Dataset_Split4/Augmented_Train"

augmenter = ImageDataGenerator(
    rotation_range=10,
    shear_range=0.2,
    zoom_range=0.2,
    fill_mode='nearest'
)

img_count = 10

def augment_participant_data(src_dir, dest_dir, category, participant_id, img_count=5):
    category_folder_path = os.path.join(dest_dir, category)
    os.makedirs(category_folder_path, exist_ok=True)

    for i in range(1, img_count + 1):
        aug_folder_name = f"Aug{i}_{participant_id}"
        aug_folder_path = os.path.join(category_folder_path, aug_folder_name)
        os.makedirs(aug_folder_path, exist_ok=True)

        for img_name in os.listdir(src_dir):
            if img_name.lower().endswith((".jpg", ".jpeg", ".png")):
                img_path = os.path.join(src_dir, img_name)
                img = load_img(img_path)
                img_array = img_to_array(img)
                img_array = img_array.reshape((1,) + img_array.shape)

                aug_iter = augmenter.flow(
                    img_array,
                    batch_size=1,
                    save_to_dir=aug_folder_path,
                    save_prefix=f"aug_{os.path.splitext(img_name)[0]}",
                    save_format="jpeg"
                )
                next(aug_iter)

for category in os.listdir(train_dir):
    category_path = os.path.join(train_dir, category)
    if os.path.isdir(category_path):
        for participant_id in os.listdir(category_path):
            participant_path = os.path.join(category_path, participant_id)
            if os.path.isdir(participant_path):
                augment_participant_data(
                    src_dir=participant_path,
                    dest_dir=augmented_dir,
                    category=category,
                    participant_id=participant_id,
                    img_count=img_count
                )

print("Data augmentation completed!")


Data augmentation completed!


In [ ]:
import os
import torch
from PIL import Image
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader

class SimpleImageDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        for category in os.listdir(root_dir):
            category_dir = os.path.join(root_dir, category)
            if os.path.isdir(category_dir):
                for participant_folder in os.listdir(category_dir):
                    participant_path = os.path.join(category_dir, participant_folder)
                    if os.path.isdir(participant_path):
                        for img_name in os.listdir(participant_path):
                            if img_name.lower().endswith((".jpg", ".jpeg", ".png")):
                                self.image_paths.append(os.path.join(participant_path, img_name))

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image

augmented_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

train_dir = "/content/drive/MyDrive/ENPM703/Project/Final/Dataset3/Dataset_Split2/Augmented_Train"

dataset = SimpleImageDataset(train_dir, transform=augmented_transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=False, num_workers=2)

mean_sum = torch.zeros(3)
mean_sq_sum = torch.zeros(3)
count = 0

for images in dataloader:
    batch_size = images.size(0)
    images = images.view(batch_size, 3, -1)
    mean_sum += images.mean(dim=[0, 2]) * batch_size
    mean_sq_sum += (images ** 2).mean(dim=[0, 2]) * batch_size
    count += batch_size

mean = mean_sum / count
mean_sq = mean_sq_sum / count
std = torch.sqrt(mean_sq - mean**2)

print("Calculated mean: ", mean)
print("Calculated std: ", std)


Calculated mean:  tensor([0.0039, 0.0039, 0.0035])
Calculated std:  tensor([0.0222, 0.0246, 0.0224])
